In [ ]:
# Purpose of this notebook:
# Plot polar amplification factor, expressed as a ratio of Arctic to global warming, from WACCM output. Also,
# perform statistical tests to see when it is greater than 0.